In [1]:
import pandas as pd
import gc

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error 

In [4]:
import pickle

In [5]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('Taxi_predictions')

<Experiment: artifact_location='/Users/davidgillespie/projects/mlops_course/week 2/course_work/mlruns/1', creation_time=1716153557618, experiment_id='1', last_update_time=1716153557618, lifecycle_stage='active', name='Taxi_predictions', tags={}>

In [6]:
def read_dataframe(filepath):
    if filepath.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    else:
        df = pd.read_parquet(filepath)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    # df['duration'] = df['duration'].dt.

    df = df[(df['duration']>=1 ) & (df['duration']<=60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    df[categorical] = df[categorical].astype(str)
    return df

In [7]:
df_train = read_dataframe('data/green_taxi_data.parquet')
df_val = read_dataframe('data/green_taxi_data_02.parquet')

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target= 'duration'
y_train = df_train[target]
y_val = df_val[target]

In [11]:
with mlflow.start_run():
    
    mlflow.set_tag("Developer", "Bob")
    mlflow.log_param("train-data-path", 'data/green_taxi_data.parquet' )
    mlflow.log_param("val-data-path", 'data/green_taxi_data_02.parquet')
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## xgBoost and Hyperparam tuning

In [12]:
import xgboost as xgb

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [15]:
def objective(params):
    with mlflow.start_run():
    
        mlflow.set_tag("model", "XgBoost")
        mlflow.log_params(params)
        
        booster = xgb.train(
            params = params,
            dtrain=train,
            num_boost_round=1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds=50,
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}


In [19]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4,100, 1)),
    'learning_rate': hp.lognormal('learning_rate', -3,0), #exp(-3), exp(0)
    'reg_alpha': hp.lognormal('reg_alpha', -5,1),
    'reg_lambda': hp.lognormal('reg_lambda', -6,1),
    'min_child_weight': hp.lognormal('min_child_weight', -1, 3),
    'objective':'reg:linear',
    'seed':42
}


In [20]:
gc.collect()

288

In [22]:
best_match = fmin(fn=objective,
      space = search_space,
      algo=tpe.suggest,
        max_evals=1,
        trials=Trials()
            )

  0%|                                     | 0/1 [00:00<?, ?trial/s, best loss=?]

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:56:51] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.81445                                                    
[1]	validation-rmse:11.44165                                                    
[2]	validation-rmse:11.09247                                                    
[3]	validation-rmse:10.76779                                                    
[4]	validation-rmse:10.46321                                                    
[5]	validation-rmse:10.18011                                                    
[6]	validation-rmse:9.91465                                                     
[7]	validation-rmse:9.66990                                                     
[8]	validation-rmse:9.44269                                                     
[9]	validation-rmse:9.23056                                                     
[10]	validation-rmse:9.03304                                                    
[11]	validation-rmse:8.85061                                                    
[12]	validation-rmse:8.68017

[202]	validation-rmse:6.55127                                                   
[203]	validation-rmse:6.55114                                                   
[204]	validation-rmse:6.55092                                                   
[205]	validation-rmse:6.55073                                                   
[206]	validation-rmse:6.55045                                                   
[207]	validation-rmse:6.55020                                                   
[208]	validation-rmse:6.55007                                                   
[209]	validation-rmse:6.54983                                                   
[210]	validation-rmse:6.54968                                                   
[211]	validation-rmse:6.54947                                                   
[212]	validation-rmse:6.54907                                                   
[213]	validation-rmse:6.54897                                                   
[214]	validation-rmse:6.5486

[404]	validation-rmse:6.51179                                                   
[405]	validation-rmse:6.51160                                                   
[406]	validation-rmse:6.51131                                                   
[407]	validation-rmse:6.51123                                                   
[408]	validation-rmse:6.51100                                                   
[409]	validation-rmse:6.51087                                                   
[410]	validation-rmse:6.51090                                                   
[411]	validation-rmse:6.51061                                                   
[412]	validation-rmse:6.51031                                                   
[413]	validation-rmse:6.51023                                                   
[414]	validation-rmse:6.51000                                                   
[415]	validation-rmse:6.50989                                                   
[416]	validation-rmse:6.5097

[606]	validation-rmse:6.48268                                                   
[607]	validation-rmse:6.48267                                                   
[608]	validation-rmse:6.48238                                                   
[609]	validation-rmse:6.48215                                                   
[610]	validation-rmse:6.48197                                                   
[611]	validation-rmse:6.48182                                                   
[612]	validation-rmse:6.48171                                                   
[613]	validation-rmse:6.48109                                                   
[614]	validation-rmse:6.48101                                                   
[615]	validation-rmse:6.48087                                                   
[616]	validation-rmse:6.48071                                                   
[617]	validation-rmse:6.48056                                                   
[618]	validation-rmse:6.4804

[808]	validation-rmse:6.46240                                                   
[809]	validation-rmse:6.46228                                                   
[810]	validation-rmse:6.46232                                                   
[811]	validation-rmse:6.46218                                                   
[812]	validation-rmse:6.46226                                                   
[813]	validation-rmse:6.46229                                                   
[814]	validation-rmse:6.46220                                                   
[815]	validation-rmse:6.46218                                                   
[816]	validation-rmse:6.46203                                                   
[817]	validation-rmse:6.46180                                                   
[818]	validation-rmse:6.46183                                                   
[819]	validation-rmse:6.46179                                                   
[820]	validation-rmse:6.4618

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [23]:
mlflow.xgboost.autolog(disable=True)

In [24]:
best_match

{'learning_rate': 0.049787068367863944,
 'max_depth': 29.0,
 'min_child_weight': 0.272047137832333,
 'reg_alpha': 0.004088654350520683,
 'reg_lambda': 0.010092653929616738}

In [25]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:58:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

[277]	validation-rmse:6.40666
[278]	validation-rmse:6.40632
[279]	validation-rmse:6.40581
[280]	validation-rmse:6.40552
[281]	validation-rmse:6.40505
[282]	validation-rmse:6.40472
[283]	validation-rmse:6.40449
[284]	validation-rmse:6.40420
[285]	validation-rmse:6.40377
[286]	validation-rmse:6.40347
[287]	validation-rmse:6.40305
[288]	validation-rmse:6.40283
[289]	validation-rmse:6.40268
[290]	validation-rmse:6.40245
[291]	validation-rmse:6.40219
[292]	validation-rmse:6.40207
[293]	validation-rmse:6.40178
[294]	validation-rmse:6.40135
[295]	validation-rmse:6.40101
[296]	validation-rmse:6.40064
[297]	validation-rmse:6.40041
[298]	validation-rmse:6.40004
[299]	validation-rmse:6.39995
[300]	validation-rmse:6.39958
[301]	validation-rmse:6.39939
[302]	validation-rmse:6.39900
[303]	validation-rmse:6.39883
[304]	validation-rmse:6.39869
[305]	validation-rmse:6.39846
[306]	validation-rmse:6.39837
[307]	validation-rmse:6.39819
[308]	validation-rmse:6.39793
[309]	validation-rmse:6.39771
[310]	vali

[551]	validation-rmse:6.35636
[552]	validation-rmse:6.35630
[553]	validation-rmse:6.35613
[554]	validation-rmse:6.35621
[555]	validation-rmse:6.35604
[556]	validation-rmse:6.35586
[557]	validation-rmse:6.35563
[558]	validation-rmse:6.35564
[559]	validation-rmse:6.35548
[560]	validation-rmse:6.35528
[561]	validation-rmse:6.35488
[562]	validation-rmse:6.35475
[563]	validation-rmse:6.35457
[564]	validation-rmse:6.35443
[565]	validation-rmse:6.35436
[566]	validation-rmse:6.35431
[567]	validation-rmse:6.35407
[568]	validation-rmse:6.35379
[569]	validation-rmse:6.35378
[570]	validation-rmse:6.35371
[571]	validation-rmse:6.35347
[572]	validation-rmse:6.35313
[573]	validation-rmse:6.35301
[574]	validation-rmse:6.35292
[575]	validation-rmse:6.35259
[576]	validation-rmse:6.35244
[577]	validation-rmse:6.35232
[578]	validation-rmse:6.35216
[579]	validation-rmse:6.35197
[580]	validation-rmse:6.35184
[581]	validation-rmse:6.35171
[582]	validation-rmse:6.35158
[583]	validation-rmse:6.35155
[584]	vali

[825]	validation-rmse:6.32990
[826]	validation-rmse:6.32972
[827]	validation-rmse:6.32970
[828]	validation-rmse:6.32958
[829]	validation-rmse:6.32960
[830]	validation-rmse:6.32935
[831]	validation-rmse:6.32935
[832]	validation-rmse:6.32924
[833]	validation-rmse:6.32914
[834]	validation-rmse:6.32909
[835]	validation-rmse:6.32881
[836]	validation-rmse:6.32883
[837]	validation-rmse:6.32881
[838]	validation-rmse:6.32873
[839]	validation-rmse:6.32861
[840]	validation-rmse:6.32864
[841]	validation-rmse:6.32855
[842]	validation-rmse:6.32851
[843]	validation-rmse:6.32846
[844]	validation-rmse:6.32829
[845]	validation-rmse:6.32817
[846]	validation-rmse:6.32795
[847]	validation-rmse:6.32786
[848]	validation-rmse:6.32772
[849]	validation-rmse:6.32758
[850]	validation-rmse:6.32751
[851]	validation-rmse:6.32749
[852]	validation-rmse:6.32739
[853]	validation-rmse:6.32729
[854]	validation-rmse:6.32725
[855]	validation-rmse:6.32729
[856]	validation-rmse:6.32718
[857]	validation-rmse:6.32688
[858]	vali

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:59:05] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [11]:
import setuptools
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", 'data/green_taxi_data.parquet' )
        mlflow.log_param("val-data-path", 'data/green_taxi_data_02.parquet')
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2024/05/19 23:11:29 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2024/05/19 23:21:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/05/19 23:21:11 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squ